In [19]:
# from comet_ml import Experiment
# experiment = Experiment(api_key="xktj4EX0zB8YcQ3BEaFwOQYpu")

In [20]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.regularizers import l2
from keras.utils import np_utils 

In [35]:
import os
import fnmatch

import pandas as pd

import sklearn.preprocessing as preprocessing
from sklearn.cross_validation import StratifiedKFold

C:\Users\Duo\Anaconda3\envs\fastai\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [22]:
train_path = "I:\\Honours-Project\\data\\sorted\\agg\\train\\"
train_csv = "I:\\Honours-Project\\data\\sorted\\agg\\train\\ids.csv"
test_path = "I:\\Honours-Project\\data\\sorted\\agg\\test\\"
test_csv = "I:\\Honours-Project\\data\\sorted\\agg\\test\\ids.csv"

dest_path = "I:\\Honours-Project\\data\\sorted\\agg\\"

csv = "I:\\Honours-Project\\data\\sorted\\agg\\ids.csv"

In [23]:
df = pd.read_csv(csv,index_col=0).sample(frac=1)

df['log_volume'] = df['volume'].apply(np.log)

x = df.drop(columns=['name','date','flow_rate','source','moments','inertia_tensor'])
x = standardize = preprocessing.scale(x)

In [24]:
df.head()

,name,date,flow_rate,source,peak_val,area,volume,eccentricity,euler_number,extent,mean_intensity,moments,orientation,perimeter,inertia_tensor,cluster,num_peaks,log_volume
146,146,2017.11.23,1800,63.190 s.tifheightmap.mat,9.263165,7400,25290.418483,0.964196,0,0.252870,3.417162,[[7.40100000e+03 1.12974800e+06 2.06903164e+08...,-1.311872,796.139177,[[ 650.68892314 -1140.5592428 ]\r\n [-1140.55...,2,2,10.138181
306,193_flipped,2017.11.23,7200,17.00 s.tifheightmap.mat,22.020838,8174,90861.108863,0.866445,0,0.612744,11.115868,[[8.17400000e+03 5.57136000e+05 4.89365720e+07...,-1.238136,482.315801,[[ 480.1139295 -337.80767374]\r\n [-337.80767...,0,1,11.417087
356,58_flipped,2017.11.17,7200,21.65 s.tifheightmap.mat,6.398700,2113,8579.705339,0.926920,1,0.772006,4.009208,[[2.14000000e+03 8.31620000e+04 4.24636000e+06...,1.538944,206.568542,[[ 67.23740305 12.97797057]\r\n [ 12.97797057...,2,1,9.057155
70,70,2017.11.17,7200,3.70 s.tifheightmap.mat,6.696278,6441,24631.191797,0.976499,1,0.692134,3.824125,[[6.44100000e+03 6.26642000e+05 7.64434660e+07...,-1.531355,447.663997,[[ 115.35188709 -90.41671324]\r\n [ -90.41671...,2,1,10.111769
239,132_flipped,2017.11.23,1800,26.20 s.tifheightmap.mat,9.358412,5355,24621.497101,0.668227,0,0.767742,4.597852,[[5.35500000e+03 2.30840000e+05 1.30236900e+07...,-1.151411,335.622366,[[ 388.86804988 -102.91215484]\r\n [-102.91215...,0,1,10.111375


In [25]:
test_size = int(df.shape[0]/10)

x_test = np.array(x[:test_size])
y_test = df['flow_rate'][:test_size].values

x_train = np.array(x[test_size:])
y_train = df['flow_rate'][test_size:].values

In [26]:
print(np.bincount(y_train))
np.bincount(y_test)

[  0   0   0 ...   0   0 103]


array([ 0,  0,  0, ...,  0,  0, 13], dtype=int64)

## Run Logistic Regression

In [27]:
batch_size = 16
nb_classes = 3
nb_epoch = 20

In [28]:
input_dim = x_train.shape[1]

In [29]:
y_train[y_train == 1800] = 0
y_train[y_train == 3600] = 1
y_train[y_train == 7200] = 2

y_test[y_test == 1800] = 0
y_test[y_test == 3600] = 1
y_test[y_test == 7200] = 2

y_train = np_utils.to_categorical(y_train, nb_classes) 
y_test = np_utils.to_categorical(y_test, nb_classes)

In [62]:
lmda = 0.01

def build_logistic_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(output_dim, input_dim=input_dim,
                    kernel_regularizer=l2(lmda),
                    activation='softmax'))

    return model

In [31]:
model = None
model = build_logistic_model(input_dim, nb_classes)

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 3)                 39        
Total params: 39
Trainable params: 39
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [33]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 362 samples, validate on 40 samples
Epoch 1/20
362/362 [==============================] - 0s 784us/step - loss: 1.7984 - acc: 0.3094 - val_loss: 1.6247 - val_acc: 0.3750
Epoch 2/20
362/362 [==============================] - 0s 347us/step - loss: 1.6866 - acc: 0.3122 - val_loss: 1.5525 - val_acc: 0.4000
Epoch 3/20
362/362 [==============================] - 0s 358us/step - loss: 1.5964 - acc: 0.3204 - val_loss: 1.4906 - val_acc: 0.4000
Epoch 4/20
362/362 [==============================] - 0s 351us/step - loss: 1.5194 - acc: 0.3260 - val_loss: 1.4403 - val_acc: 0.3750
Epoch 5/20
362/362 [==============================] - 0s 351us/step - loss: 1.4557 - acc: 0.3149 - val_loss: 1.3983 - val_acc: 0.4000
Epoch 6/20
362/362 [==============================] - 0s 368us/step - loss: 1.4031 - acc: 0.3177 - val_loss: 1.3636 - val_acc: 0.4500
Epoch 7/20
362/362 [==============================] - 0s 270us/step - loss: 1.3582 - acc: 0.3287 - val_loss: 1.3329 - val_acc: 0.4250
Epoch 8/20
362/36

In [34]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.1484870433807373
Test accuracy: 0.4


## With Cross Validation

In [51]:
batch_size = 16
nb_classes = 3
nb_epoch = 20

In [56]:
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

In [63]:
n_folds = 10

labels = df['flow_rate'].values
data = x

labels[labels == 1800] = 0
labels[labels == 3600] = 1
labels[labels == 7200] = 2

labels = np_utils.to_categorical(labels, nb_classes) 

skf = StratifiedKFold(df['flow_rate'].values, n_folds=n_folds, shuffle=True)
avg_acc = 0

for i, (train, test) in enumerate(skf):
    print("Running Fold", i+1, "/", n_folds)
    model = None # Clearing the NN.
    model = build_logistic_model(input_dim, nb_classes)
    avg_acc += train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])

Running Fold 1 / 10
Train on 360 samples, validate on 42 samples
Epoch 1/20
360/360 [==============================] - 1s 3ms/step - loss: 1.4569 - acc: 0.2972 - val_loss: 1.4082 - val_acc: 0.3571
Epoch 2/20
360/360 [==============================] - 0s 290us/step - loss: 1.3820 - acc: 0.3111 - val_loss: 1.3478 - val_acc: 0.3095
Epoch 3/20
360/360 [==============================] - 0s 264us/step - loss: 1.3265 - acc: 0.3139 - val_loss: 1.2979 - val_acc: 0.3095
Epoch 4/20
360/360 [==============================] - 0s 234us/step - loss: 1.2820 - acc: 0.3278 - val_loss: 1.2627 - val_acc: 0.2857
Epoch 5/20
360/360 [==============================] - 0s 278us/step - loss: 1.2505 - acc: 0.3361 - val_loss: 1.2378 - val_acc: 0.2857
Epoch 6/20
360/360 [==============================] - 0s 320us/step - loss: 1.2254 - acc: 0.3389 - val_loss: 1.2200 - val_acc: 0.3095
Epoch 7/20
360/360 [==============================] - 0s 322us/step - loss: 1.2041 - acc: 0.3472 - val_loss: 1.2029 - val_acc: 0.3095

361/361 [==============================] - 0s 278us/step - loss: 1.0819 - acc: 0.4211 - val_loss: 1.1106 - val_acc: 0.4146
Running Fold 4 / 10
Train on 361 samples, validate on 41 samples
Epoch 1/20
361/361 [==============================] - 1s 3ms/step - loss: 1.4909 - acc: 0.3213 - val_loss: 1.3808 - val_acc: 0.3415
Epoch 2/20
361/361 [==============================] - 0s 511us/step - loss: 1.3758 - acc: 0.3269 - val_loss: 1.2832 - val_acc: 0.3659
Epoch 3/20
361/361 [==============================] - 0s 555us/step - loss: 1.2890 - acc: 0.3269 - val_loss: 1.2252 - val_acc: 0.3659
Epoch 4/20
361/361 [==============================] - 0s 511us/step - loss: 1.2338 - acc: 0.3463 - val_loss: 1.1895 - val_acc: 0.3415
Epoch 5/20
361/361 [==============================] - 0s 511us/step - loss: 1.1982 - acc: 0.3518 - val_loss: 1.1672 - val_acc: 0.4146
Epoch 6/20
361/361 [==============================] - 0s 555us/step - loss: 1.1739 - acc: 0.3629 - val_loss: 1.1520 - val_acc: 0.4634
Epoch 7/20

362/362 [==============================] - 0s 319us/step - loss: 1.1129 - acc: 0.4006 - val_loss: 1.2157 - val_acc: 0.4000
Epoch 20/20
362/362 [==============================] - 0s 320us/step - loss: 1.1074 - acc: 0.3978 - val_loss: 1.2080 - val_acc: 0.4000
Running Fold 7 / 10
Train on 363 samples, validate on 39 samples
Epoch 1/20
363/363 [==============================] - 1s 3ms/step - loss: 1.3905 - acc: 0.3306 - val_loss: 1.3780 - val_acc: 0.3333
Epoch 2/20
363/363 [==============================] - 0s 447us/step - loss: 1.3461 - acc: 0.3444 - val_loss: 1.3381 - val_acc: 0.3333
Epoch 3/20
363/363 [==============================] - 0s 554us/step - loss: 1.3110 - acc: 0.3499 - val_loss: 1.3055 - val_acc: 0.3590
Epoch 4/20
363/363 [==============================] - 0s 464us/step - loss: 1.2815 - acc: 0.3471 - val_loss: 1.2779 - val_acc: 0.3590
Epoch 5/20
363/363 [==============================] - 0s 422us/step - loss: 1.2564 - acc: 0.3278 - val_loss: 1.2555 - val_acc: 0.4359
Epoch 6/2

363/363 [==============================] - 0s 408us/step - loss: 1.0989 - acc: 0.4215 - val_loss: 1.1653 - val_acc: 0.3333
Epoch 19/20
363/363 [==============================] - 0s 275us/step - loss: 1.0937 - acc: 0.4270 - val_loss: 1.1563 - val_acc: 0.3333
Epoch 20/20
363/363 [==============================] - 0s 276us/step - loss: 1.0888 - acc: 0.4463 - val_loss: 1.1486 - val_acc: 0.3590
Running Fold 10 / 10
Train on 363 samples, validate on 39 samples
Epoch 1/20
363/363 [==============================] - 1s 3ms/step - loss: 1.3312 - acc: 0.3912 - val_loss: 1.3676 - val_acc: 0.3846
Epoch 2/20
363/363 [==============================] - 0s 275us/step - loss: 1.2816 - acc: 0.3884 - val_loss: 1.3276 - val_acc: 0.3846
Epoch 3/20
363/363 [==============================] - 0s 406us/step - loss: 1.2431 - acc: 0.3829 - val_loss: 1.2974 - val_acc: 0.3590
Epoch 4/20
363/363 [==============================] - 0s 464us/step - loss: 1.2135 - acc: 0.3939 - val_loss: 1.2723 - val_acc: 0.3590
Epoch 5

In [64]:
print("Average Score: ", avg_acc/n_folds)

Average Score:  0.38808853851938646
